# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sklearn
print(sklearn.__version__)

## Importing Datasets

In [ ]:
import os

# Get the absolute path of the current working directory
current_directory = os.getcwd()

# Construct the relative path to your data folder and the Excel file
relative_path = 'data/laptop_data.xlsx' ## NOTE HERE I TRIED WITH OUR ALREADY CLEANED DATA BUT REALISTICALLY, WE REMOVE OUTLIERS FROM OUR DATA WHEN THEY ARE NOT YET CLEANED UP

# Combine the current directory with the relative path to get the absolute path to your Excel file
file_path = os.path.join(current_directory, relative_path)

# Read the Excel file using the constructed file path
dataset = pd.read_excel(file_path)

In [ ]:
dataset.shape

## Select the important columns necessary for training/building our model

In [ ]:
dataset = dataset[['BRAND', 'CPU BRAND', 'CPU CORE', 'CPU GENERATION', 'CPU FAMILY', 'RAM SIZE', 'RAM(DDR) TYPE', 'DISK TYPE', 'SSD SIZE',
                   'HDD SIZE', 'GPU BRAND', 'GPU TYPE', 'SCREEN SIZE', 'SCREEN RESOLUTION', 'STATE', 'PRICE']]

## Convert the DataType of categorical datas

In [ ]:
## CATEGORICAL
dataset["BRAND"] = dataset["BRAND"].astype("str")
dataset["CPU BRAND"] = dataset["CPU BRAND"].astype("str")
dataset["CPU CORE"] = dataset["CPU CORE"].astype("str")
dataset["CPU FAMILY"] = dataset["CPU FAMILY"].astype("str")
dataset["DISK TYPE"] = dataset["DISK TYPE"].astype("str")
dataset["GPU BRAND"] = dataset["GPU BRAND"].astype("str")
dataset["GPU TYPE"] = dataset["GPU TYPE"].astype("str")
dataset["SCREEN RESOLUTION"] = dataset["SCREEN RESOLUTION"].astype("str")
dataset["STATE"] = dataset["STATE"].astype("str")

In [ ]:
type(dataset)

In [ ]:
dataset.values

## Selecting the X and y columns

- **X**: represent our FEATURES that our model will learn from

- **y**: represent the Label column our model will train with

In [ ]:
X = dataset.drop('PRICE', axis = 'columns')

In [ ]:
X.shape

In [ ]:
y = dataset.PRICE

In [ ]:
y = y.values.reshape((254,1))
y.shape

## Splitting data into Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
categorical_data = dataset[['BRAND', 'CPU BRAND', 'CPU CORE', 'CPU FAMILY', 'DISK TYPE',
               'GPU BRAND', 'GPU TYPE', 'SCREEN RESOLUTION', 'STATE']]

In [ ]:
categorical_columns = ['BRAND', 'CPU BRAND', 'CPU CORE', 'CPU FAMILY', 'DISK TYPE',
               'GPU BRAND', 'GPU TYPE', 'SCREEN RESOLUTION', 'STATE']

In [ ]:
numerical_data = dataset[['CPU GENERATION', 'RAM SIZE', 'RAM(DDR) TYPE', 'SSD SIZE', 'HDD SIZE', 'SCREEN SIZE']]

In [ ]:
numerical_columns = ['CPU GENERATION', 'RAM SIZE', 'RAM(DDR) TYPE', 'SSD SIZE', 'HDD SIZE', 'SCREEN SIZE']

## Importing Libraries

In [ ]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import MinMaxScaler

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

## Normalization and Standardization

In [ ]:
num_pipeline = Pipeline([
    ("MinMax_Scaler", MinMaxScaler()),
    ('Standard_Scaler', StandardScaler())
])

## ColumnTransformer and OneHotEncoding

In [ ]:
column_trans = ColumnTransformer(transformers = [
    ("numerical", num_pipeline, numerical_columns),
    ("categorical", OneHotEncoder(sparse_output = False, handle_unknown='ignore'), categorical_columns),
    ], remainder = 'passthrough')

## Model Building

In [ ]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error

In [ ]:
from time import time

from sklearn.linear_model import RidgeCV

In [ ]:
regressors = [
    RidgeCV()
]

In [ ]:
# head = 10
for model in regressors:    # [:head]:
    start = time()
    
    
    pipe = Pipeline([
        ('step1', column_trans),
        ('step2', model)
    ])
    
    
    pipe.fit(X_train, y_train)


    train_time = time() - start
    start = time()
    predict_time = time()-start
    
    y_prediction = pipe.predict(X_test)
    
    print(model)
    print("\t Training Time: %0.3fs" % train_time)
    print("\t Prediction Time: %0.3fs" % predict_time)
    print("\t Explained Variance Score:", explained_variance_score(y_test, y_prediction))
    print("\t Mean Absolute Error:", mean_absolute_error(y_test, y_prediction))
    print("\t Mean Squared Error:", mean_squared_error(y_test, y_prediction))
    print("\t Root Mean Squared Error:", mean_squared_error(y_test, y_prediction, squared = False))
    print("\t R2 Score:", r2_score(y_test, y_prediction))
    print()

## Model Tuning with HyperParameters

In [ ]:
# Define a range of alpha values to search
alphas = np.logspace(-3, 3, 7)  # You can adjust the range as needed

# Define other parameters you want to tune
fit_intercept_values = [True, False]  # Whether to fit the intercept
normalize_values = [True, False]      # Whether to normalize the data

# Create a parameter grid
params_grid = {
    'alphas': [alphas],                   # List of alpha values
    'fit_intercept': fit_intercept_values, # List of fit_intercept values
}

ridgeCV = RidgeCV()

step2 = GridSearchCV(ridgeCV, params_grid, cv = 5, scoring = 'r2')

pipe = Pipeline([
    ('step1', column_trans),
    ('step2', step2)
])

pipe.fit(X_train,y_train)

y_prediction = pipe.predict(X_test)

print('R2 score: ', r2_score(y_test, y_prediction))
print('MAE: ', mean_absolute_error(y_test, y_prediction))
print('R-MSE: ', mean_squared_error(y_test, y_prediction, squared = False))

In [ ]:
options = {
    'BRAND': 'HP',
    'CPU BRAND': 'Intel',
    'CPU CORE': 'Core i7',
    'CPU GENERATION': 7,
    'CPU FAMILY': 'U',
    'RAM SIZE': 16,
    'RAM(DDR) TYPE': 3,
    'DISK TYPE': 'BOTH',
    'SSD SIZE': 512,
    'HDD SIZE': 500,
    'GPU BRAND': 'NVIDIA',
    'GPU TYPE': 'GeForce GTX 1050',
    'SCREEN SIZE': 14.0,
    'SCREEN RESOLUTION': 'HD',
    'STATE': 'NEW'
}

laptop_specs = pd.DataFrame(data = [options])
laptop_specs

In [ ]:
prediction = pipe.predict(laptop_specs)
prediction_rounded = prediction.round()
print("Predicted Price: ", prediction_rounded, "DA")

## Exporting the model with Pickle

In [ ]:
# import pickle
# 
# pickle.dump(pipe, open('/Users/sunnythesage/DataSpellProjects/LAPPY-PRED MODEL/Api/model.pkl', 'wb'))
# 
# print("MODEL SUCCESSFULLY EXPORTED!!!")